Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
import timeit
from six.moves import cPickle as pickle

First reload the data we generated in `1_notmnist.ipynb`.

# uncleaned train, valid and test sets
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

# cleaned train, valid and test sets
pickle_file = 'notMNIST_c.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset_c']
  train_labels = save['train_labels_c']
  valid_dataset = save['valid_dataset_c']
  valid_labels = save['valid_labels_c']
  test_dataset = save['test_dataset_c']
  test_labels = save['test_labels_c']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

In [2]:
# cleaned training, unclean valid and test
pickle_file1 = 'notMNIST_c.pickle'

with open(pickle_file1, 'rb') as f1:
  save = pickle.load(f1)
  train_dataset = save['train_dataset_c']
  train_labels = save['train_labels_c']
  del save
  

pickle_file2 = 'notMNIST.pickle'

with open(pickle_file2, 'rb') as f2:
  save = pickle.load(f2)
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
    
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (145051, 28, 28) (145051,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (145051, 784) (145051, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [5]:
# meta parameters
batch_size = 4096 #128
n_hidden = 1024 #1024
num_steps = 10001 #40001
L2_coef = 0.001 #0.001
init_learn_rate = 0.1 #0.1
final_learn_rate = 0.00015 #0.00015
momentum = .25 #0.25

init_mean = 0 # 0
standard_dev = .1 # .1
bias_init = .2 #.01

# calc learning rate decay

steps_per_batch = np.ceil(train_dataset.shape[0]/batch_size)
loops_over_data = np.floor(num_steps/steps_per_batch)

# init_learn_rate*x**loops_over_data = 0.01
# x = (.01/init_learn_rate)**(1/loops_over_data)

decay_rate = (final_learn_rate/init_learn_rate)**(1/loops_over_data) # decay rate after every loop through data, should be about .7 for 10 loops
print(decay_rate)

# create graph

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(tf.truncated_normal([image_size * image_size, n_hidden],init_mean,standard_dev))
  biases = tf.Variable(tf.zeros([n_hidden]))
  global_step = tf.Variable(0) 
  learning_rate = tf.train.exponential_decay(init_learn_rate, global_step, steps_per_batch, decay_rate, staircase=True)
    
  weights2 = tf.Variable(tf.truncated_normal([n_hidden, num_labels]))
  #biases2 = tf.Variable(tf.zeros([num_labels]))
  biases2 = tf.Variable(bias_init * tf.ones([num_labels]))

  
    
    # Training computation.
  #relu_layer_dropout = tf.nn.dropout(2*tf.nn.relu(tf.matmul(tf_train_dataset,weights)+biases),.5)
  relu_layer_dropout = tf.nn.dropout(2*tf.nn.relu(tf.matmul(tf_train_dataset,weights)+biases),.5)
  logits = tf.matmul(relu_layer_dropout, weights2) + biases2

  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))+L2_coef*tf.nn.l2_loss(weights)+L2_coef*tf.nn.l2_loss(weights2)
  
  # Optimizer.
  #optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)
  #optimizer = tf.train.AdagradOptimizer(learning_rate).minimize(loss)
  optimizer = tf.train.MomentumOptimizer(learning_rate, momentum).minimize(loss)
  
  # Predictions for the training, validation, and test data.
    
  relu_layer_NO_dropout = tf.nn.relu(tf.matmul(tf_train_dataset,weights)+biases)
  logits_NO_droput = tf.matmul(relu_layer_NO_dropout, weights2) + biases2
  train_prediction = tf.nn.softmax(logits_NO_droput)
  relu_layer_valid = tf.nn.relu(tf.matmul(tf_valid_dataset,weights)+biases)
  valid_prediction = tf.nn.softmax(tf.matmul(relu_layer_valid, weights2) + biases2)
  relu_layer_test = tf.nn.relu(tf.matmul(tf_test_dataset,weights)+biases)
  test_prediction = tf.nn.softmax(tf.matmul(relu_layer_test, weights2) + biases2)

0.976799398731


In [6]:
start_time = timeit.default_timer()

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    ##offset = (step % 2) * batch_size # this causes overfitting
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

elapsed = timeit.default_timer() - start_time
print("Elapsed time: %.0f seconds" % elapsed)

Initialized
Minibatch loss at step 0: 106.982796
Minibatch accuracy: 10.8%
Validation accuracy: 49.4%
Minibatch loss at step 500: 7.479926
Minibatch accuracy: 84.2%
Validation accuracy: 85.0%
Minibatch loss at step 1000: 5.785932
Minibatch accuracy: 88.9%
Validation accuracy: 85.9%
Minibatch loss at step 1500: 4.843486
Minibatch accuracy: 91.1%
Validation accuracy: 86.8%
Minibatch loss at step 2000: 4.164562
Minibatch accuracy: 92.7%
Validation accuracy: 87.2%
Minibatch loss at step 2500: 3.648344
Minibatch accuracy: 93.3%
Validation accuracy: 87.8%
Minibatch loss at step 3000: 3.174042
Minibatch accuracy: 94.3%
Validation accuracy: 88.1%
Minibatch loss at step 3500: 2.799952
Minibatch accuracy: 94.8%
Validation accuracy: 88.2%
Minibatch loss at step 4000: 2.495526
Minibatch accuracy: 94.7%
Validation accuracy: 88.4%
Minibatch loss at step 4500: 2.211133
Minibatch accuracy: 95.4%
Validation accuracy: 88.6%
Minibatch loss at step 5000: 1.971824
Minibatch accuracy: 95.5%
Validation accur